## Discord Automation 
$
\text{\small{(sounds illegal, learning purpose only hehe)}}
$

First, import the libraries you'll need. I added "random" just to avoid constant grabbing time, randomize it, so it may looks natural (even if it's not actually).

Also, there's a file, webserver, that I found on youtube, [here](https://www.youtube.com/watch?v=GBwNQImv9Ek) and if you wanna learn from start, you can try it from [here](https://www.youtube.com/watch?v=xh28F6f-Cds) (it's the tutorial that helped me to create the Discord automation program).

In [ ]:
import requests
import os
import json
import time

import random

from webserver import keep_alive


Then, create variables called "acc_token" and "ch_id" that contain your account token and the id of the channel that you'll use, respectively. You can find your account token by following the steps in the video that I've linked before.

After that, you can define some definitions to retrieve messages and add reactions since the bot will drop the cards and put some reactions on them (there's a button feature, but now I only show you for the reaction case). The url is Discord's default url. The difference is only in the channel id for each channel you entered.

In "retrieve_messages" definition, I put a "while" loop if the status_code is equal to 429. That means there are too much requests in the request library, so to avoid the thing, I'm using "os.system("kill 1")" and requesting it again until the status_code turns to 200 (they accept my request).



In [ ]:
acc_token = os.environ['acc']
ch_id = os.environ['ch']

def retrieve_messages(acc_token=acc_token, ch_id=ch_id):
    msg_url = f"https://discord.com/api/v9/channels/{ch_id}/messages"
    r = requests.get(f'{msg_url}', headers={'authorization':acc_token})
    while r.status_code == 429:
      os.system("kill 1")
      r = requests.get(f'{msg_url}', headers={'authorization':acc_token})
    jsonn = json.loads(r.text)
    return jsonn

def add_reaction(msg_id, emoji, acc_token=acc_token):
   msg_url = f"https://discord.com/api/v9/channels/{ch_id}/messages"
   requests.put(f'{msg_url}/{msg_id}/reactions/{emoji}/%40me', headers={'authorization':acc_token})


Now, come to the main code. My program works by retrieving messages indefinitely. By reading the content message, my program will differ if the message is the message that I'm looking for. If it's the message, then the cards that the bot just dropped (the bot will drop 3 cards) may contain a high wishlist (pog) card, so I'll take the chance to grab it. Since my server has a bot that will remind you if there's a pog card or not, I added a feature to notice the things too.

Every card the bot drops will expire (you won't be able to receive the card or the other items anymore) in a minute and will be dropped in roughly 3 minutes.Since the goal of this program is to obtain the watermelon (an event feature of the bot), I let my program stop for around 4 seconds before reading the message once more to allow the bot to add reactions to the message. I do this by using the time.sleep(x second) command. Is there a watermelon reaction being added to it by the bot or not? My program will be able to tell if the number of message reactions is four (or more than three), or just three. If there are four, then there's a watermelon on it, then my program will grabit for me.

After grabbing the watermelon, I let my bot sleep for a minute, waited for the message to expire, then read the new message again until the bot dropped the cards again :)

In [ ]:
keep_alive()    # the function of webserver library, to keep my program running


while True:
    msg=retrieve_messages()[:2] # Get only two most recent messages
    pog = 0
    
    for index, current_msg in enumerate(msg):
        if 'dropping 3' in current_msg['content']:
            time.sleep(random.uniform(1,1.5))
            current_msg = retrieve_messages()[0]

            # Is ther a pog or not?
            if 'Card no' in current_msg['content']:
                print('pog detected')
                emoji = current_msg['content'].split()[3]
                time.sleep(random.uniform(0.2,0.4))
                the_msg = retrieve_messages()[1]
                add_reaction(the_msg['id'], emoji)
                pog = 1
            
            if pog == 1:    # if there's a pog card
                time.sleep(2)   # to wait the reactions fully reacted by the bot
                current_msg = retrieve_messages()[2]    # there will be a msg after I grabbed the card, that's why I get the 3rd msg instead
                if 'Extra Grab' in current_msg['content']:  # if my acc use an extra grab
                    current_msg = retrieve_messages()[3]
            else:   # if no pog detected
                time.sleep(3)   # to wait the reactions fully reacted by the bot
                current_msg = retrieve_messages()[0]

            try:
                current_msg['reactions']
            except:
                pass
            else:
                # if there are a watermelon in the reactions,  the length of
                # the reactions should be more than 3, since my server don't have
                # any blessing (the bot feature to let the bot drop 4 cards instead of 3).
                if len(current_msg['reactions']) > 3:
                    print('suika ga aru!')  # it means, there's a watermelon, but in weebs' language hehe
                    emoji = current_msg['reactions'][-1]['emoji']['name']
                    add_reaction(current_msg['id'], emoji)
            time.sleep(60)  # the message will expired after 60, and the bot will drop again after around 3 minutes


    time.sleep(2)   # the cooldown of my program to read messages


# Fin~

If you wanna let your code running, try to use [replit](https://replit.com/) (not for commercial purpose) and [uptimerobot](https://uptimerobot.com/) to maintain it.

Thanks for your attention!

Enjoy!